In [2]:
import os
import cv2
import numpy as np
import torchio as tio
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

In [3]:
BASE_DIR_SEG = '../data/MICCAI_BraTS_2019_Data_Training/'

In [4]:
print(os.listdir(BASE_DIR_SEG))

['HGG', 'LGG', 'survival_data.csv', 'name_mapping.csv']


In [8]:
data_dir = BASE_DIR_SEG + 'HGG'
Patient_info = [(f.path, f.name) for f in (os.scandir(data_dir)) if f.is_dir()]
sorted_patient_info = sorted(Patient_info, key=lambda param: param[1])
Patient_paths, Patient_names = zip(*sorted_patient_info)
print(Patient_paths)

('../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_12_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_13_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_14_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_17_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_18_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_19_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_20_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_21_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_22_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_23_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_25_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_26_1', '../data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_27_1', '../data/MICCAI_BraTS_20

In [13]:
path = Patient_paths[0]
name = Patient_names[0]
image = tio.ScalarImage(f"{path}/{name}_t1ce.nii")
image[tio.DATA] = image[tio.DATA].to(torch.float32)
label = tio.LabelMap(f"{path}/{name}_seg.nii")
print(image)
color_img = np.stack([image[0].cpu().detach().numpy()] * 3, axis=-1)

ScalarImage(shape: (1, 240, 240, 155); spacing: (1.00, 1.00, 1.00); orientation: LPS+; dtype: torch.FloatTensor; memory: 34.1 MiB)


KeyError: 0